In [1]:
import xarray
import zarr

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import dask

from datetime import datetime
from pycontrails.datalib.ecmwf import ERA5ARCO
from pycontrails.datalib.ecmwf.arco_era5 import open_arco_era5_model_level_data
from pycontrails.models.cocip import Cocip
from pycontrails.datalib.ecmwf.model_levels import model_level_pressure
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.apcemm import APCEMM
from pycontrails.core import met_var, GeoVectorDataset, models, vector
from pycontrails.physics import constants, thermo, units
from pycontrails.datalib.ecmwf import ERA5ModelLevel
from pycontrails import MetDataArray, MetDataset, MetVariable
from pycontrails import Flight
from pycontrails.models.dry_advection import DryAdvection
from pycontrails.models.humidity_scaling import ConstantHumidityScaling

In [2]:
times = pd.date_range("2020-02-25T01:00", "2020-02-25T05:00", periods=3)

ds = open_arco_era5_model_level_data(
    times=times,
    variables=Cocip.met_variables+Cocip.optional_met_variables,
    pressure_levels=(200,250,300)
)

In [3]:
ds = ds.sel(longitude=slice(45, 50))
ds = ds.sel(latitude=slice(80, 89))

ds = ds.rename({"t":"air_temperature","u":"eastward_wind","v":"northward_wind","w":"lagrangian_tendency_of_air_pressure"})

In [4]:
print(ds.nbytes)

140396


In [5]:
met = MetDataset(ds)

In [ ]:
print(met)

MetDataset with data:

<xarray.Dataset>
Dimensions:                              (time: 3, latitude: 37, level: 3,
                                          longitude: 21)
Coordinates:
  * time                                 (time) datetime64[ns] 2020-02-25T01:...
  * latitude                             (latitude) float64 80.0 80.25 ... 89.0
  * level                                (level) float64 200.0 250.0 300.0
  * longitude                            (longitude) float64 45.0 45.25 ... 50.0
    air_pressure                         (level) float32 2e+04 2.5e+04 3e+04
    altitude                             (level) float32 1.178e+04 ... 9.164e+03
Data variables:
    air_temperature                      (longitude, latitude, level, time) float32 dask.array<chunksize=(21, 37, 3, 1), meta=np.ndarray>
    q                                    (longitude, latitude, level, time) float32 dask.array<chunksize=(21, 37, 3, 1), meta=np.ndarray>
    eastward_wind                        (longit

: 

In [ ]:
with dask.config.set(scheduler="single-threaded"):
    met.data.load()

In [ ]:
flight_attrs = {
    "flight_id": "test",
    # set constants along flight path
    "true_airspeed": 226.099920796651,  # true airspeed, m/s
    "thrust": 0.22,  # thrust_setting
    "nvpm_ei_n": 1.897462e15,  # non-volatile emissions index
    "aircraft_type": "E190",
    "wingspan": 48,  # m
    "n_engine": 2,
}

# Example flight
df = pd.DataFrame()
df["longitude"] = np.linspace(47, 47, 1)
df["latitude"] = np.linspace(85, 85, 1)
df["altitude"] = np.linspace(10900, 10900, 1)
df["engine_efficiency"] = np.linspace(0.34, 0.34, 1)
df["fuel_flow"] = np.linspace(2.1, 2.1, 1)  # kg/s
df["aircraft_mass"] = np.linspace(154445, 154445, 1)  # kg
df["time"] = pd.date_range("2020-02-25T01:00", "2020-02-25T01:00", periods=1)

fl = Flight(df, attrs=flight_attrs)

dt_integration = pd.Timedelta(minutes=2)
max_age = pd.Timedelta(hours=6)

params = {
    "dt_integration": dt_integration,
    "max_age": max_age,
    "depth": 1.0,  # initial plume depth, [m]
    "width": 1.0,  # initial plume width, [m]
}
dry_adv = DryAdvection(met, params)
dry_adv_df = dry_adv.eval(fl).dataframe

ax = plt.axes()

ax.scatter(fl["longitude"], fl["latitude"], s=3, color="red", label="Flight path")
ax.scatter(
    dry_adv_df["longitude"], dry_adv_df["latitude"], s=0.1, color="purple", label="Plume evolution"
)
ax.legend()
ax.set_title("Flight path and plume evolution under dry advection")

plt.show()